In [112]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from numba import jit , cuda
from PIL import Image
import random

In [113]:
test_Inp = []
random.seed(42)
i = 0 
ts_c = 10
while(i<ts_c) : 
    x = (random.choice(range(0,19906)))
    if x not in test_Inp:
        test_Inp.append(x)
        i+=1
test_Out=[]
for i in range(0,ts_c):
    test_Out.append(test_Inp[i])
print(test_Inp)
print(test_Out)

[3648, 819, 9012, 8024, 7314, 4572, 3358, 17870, 2848, 19349]
[3648, 819, 9012, 8024, 7314, 4572, 3358, 17870, 2848, 19349]


In [32]:
Data = pd.read_csv(r"V:\python\Neural Network\Training_Dataset\train.csv")
Data["ID"]
Alpha = 0.5

In [33]:


@jit(target_backend='cuda')
def func(Image1,Data):
    data=[]
    for i in Data:
        data.append(np.asarray(Image.open(Image1+i).resize((150,150)).convert("L")))
    return data


if __name__=="__main__":
    image=[]
    Image1=r"V:\python\Neural Network\Training_Dataset\Train\\"
    image = func(Image1,Data["ID"])
    print("done")




C:\Users\VIGNESH RAAM SR\AppData\Local\Temp\ipykernel_2872\3850650326.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "func" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\VIGNESH RAAM SR\AppData\Local\Temp\ipykernel_2872\3850650326.py (3)

File "C:\Users\VIGNESH RAAM SR\AppData\Local\Temp\ipykernel_2872\3850650326.py", line 3:
def func(Image1,Data):
    data=[]
    ^

  @jit(target_backend='cuda')
C:\Users\VIGNESH RAAM SR\AppData\Local\Temp\ipykernel_2872\3850650326.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "func" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "C:\Users\VIGNESH RAAM SR\AppData\Local\Temp\ipykernel_2872\3850650326.py", line 4:
def func(Image1,Data):
    <source elided>
    data=[]
    for i in Data:
    ^

  @jit(target_backend='cuda')
c

done


In [114]:
@jit(target_backend='cuda')
def row_major_ordering(arr,x):
    for i in range(0,len(arr)):
        arr[i]=arr[i].reshape((1,x))

image_row_major_ordered = image.copy()
row_major_ordering(image_row_major_ordered,22500)
test_data = []
for i in test_Inp:
    test_data.append(image_row_major_ordered[i])
    image_row_major_ordered.pop(i)


c:\Users\VIGNESH RAAM SR\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\ir_utils.py:2147: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'arr' of function 'row_major_ordering'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "C:\Users\VIGNESH RAAM SR\AppData\Local\Temp\ipykernel_2872\411657787.py", line 2:
@jit(target_backend='cuda')
def row_major_ordering(arr,x):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


In [116]:
D = Data["Class"]
Ground_Truth = []
for i in D:
    if i=="YOUNG":
        Ground_Truth.append(np.array([1,0,0]))
    elif i=="MIDDLE":
        Ground_Truth.append(np.array([0,1,0]))
    elif i =="OLD":
        Ground_Truth.append(np.array([0,0,1]))
test_out= []
for i in test_Out:
    test_out.append(Ground_Truth[i])
    Ground_Truth.pop(i)

In [117]:
Output = np.array(Ground_Truth)
Input1 = np.array(image_row_major_ordered).reshape((19906-ts_c,22500))

In [118]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Input(shape = Input1[0].shape))
model.add(Dense(15, activation = "sigmoid"))
model.add(Dense(3, activation = "sigmoid"))

model.summary()


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 15)                337515    
                                                                 
 dense_13 (Dense)            (None, 3)                 48        
                                                                 
Total params: 337,563
Trainable params: 337,563
Non-trainable params: 0
_________________________________________________________________


In [119]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = "binary_accuracy")

In [120]:
model.fit(Input1,Output, epochs = 2)

Epoch 1/2
622/622 [==============================] - 7s 10ms/step - loss: 0.5707 - binary_accuracy: 0.6941
Epoch 2/2
622/622 [==============================] - 5s 7ms/step - loss: 0.5654 - binary_accuracy: 0.6952


In [153]:
def Output_Opt(d):
    arr=[]
    for i in range(0,3):
        if d[0][i]<0.5:
            arr.append(0)
        else:
            arr.append(1)
    return arr

In [155]:
Input2 = np.array(test_data).reshape((ts_c,22500))
count = 0
for i in range(0,len(Input2)):
    data = Output_Opt(model.predict(Input2[i].reshape((1,22500))))
    if ( list(test_out[i]) == data):
        count  = count +  1 
print("Accuracy is  : " ,(count/ts_c) * 100)
    

1/1 [==============================] - 0s 32ms/step
Accuracy is  :  50.0


Black Box Successfully Implemented